# 🚀 Extraction de pages reliées à des concepts par propriété

## 📑 Mode d'emploi

Suivre les instructions au fil du notebook et exécuter une à une les cellules de code en appuyant sur la petite flèche à gauche (▶️)

## 🔨 Construction de l'environnement nécessaire et configuration

### Installation des modules

In [ ]:
# 📦 MODULES NECESSAIRES : NORMALEMENT, NE RUN QU'A LA PREMIERE UTILISATION
%pip install -q SPARQLWrapper tqdm pandas

print("🎉 Installation terminée avec succès !")

### Configuration
#### Paramètres

In [ ]:
# 🔧 IMPORTS PYTHON

import json
import csv
import time
import re
import os
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import pandas as pd

print("✨ Modules Python importés avec succès !")

In [ ]:
# 🔧 CONFIGURATION PERSONNALISABLE DE LA REQUETE

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql" # Endpoint SPARQL de Wikidata
RATE_LIMIT_DELAY = 3.0  # Délai entre les requêtes
BATCH_SIZE = 10  # Taille des batches pour les requêtes
MAX_RETRIES = 3  # Nombre maximal de tentatives en cas d'échec
REQUEST_TIMEOUT = 60 # Temps au bout duquel un requête s'arrête automatiquement s'il n'y a pas de réponse (en secondes)
ENRICHMENT_BATCH_SIZE = 15 # Taille du batch pour l'enrichissement des données
LOOP_LIMIT = 100 # Nombre réponses limite par boucle (permet de requêter petit à petit pour ne pas surcharger l'API)
LOOP_OFFSET = 0 # Décalage pour la pagination des résultats
MAX_RESULTS = None  # Nombre maximal de résultats à récupérer (pour éviter de surcharger l'API)


# 📁 Configuration du dossier de sortie 
output_dir = "./QUERY_BY_PROP_output" # Paramètre à remplacer si vous souhaitez un autre dossier de sortie (faire un copier coller du chemin d'un fichier)
os.makedirs(output_dir, exist_ok=True)

print("🎯" + "="*98 + "🎯")
print("🚀 CONFIGURATION SYSTÈME TERMINÉE 🚀".center(100))
print("🎯" + "="*98 + "🎯")
print(
    f"📂 Dossier de sortie                  📁 {output_dir}\n"
    f"⏱️  Délai entre requêtes              🕐 {RATE_LIMIT_DELAY}s\n"
    f"📦 Taille des batches                 📊 {BATCH_SIZE} éléments\n"
    f"🔄 Tentatives maximales               🎯 {MAX_RETRIES} essais\n"
    f"⏳ Timeout des requêtes               ⏰ {REQUEST_TIMEOUT}s\n"
    f"🔍 Taille batch enrichissement        📈 {ENRICHMENT_BATCH_SIZE} éléments\n"
    f"🔁 Limite par boucle                  🎪 {LOOP_LIMIT} itérations\n"
    f"🌐 Endpoint Wikidata                  🔗 {WIKIDATA_ENDPOINT}"
)
print("🎯" + "="*98 + "🎯")

#### Utilitaires

In [ ]:
# 🛠️ DEFINIR LES FONCTIONS UTILITAIRES

# ================================================================================
# POUR CHOISIR UN TERME A REQUÊTER
# ================================================================================

def ask_query_term():
    """
    Fonction pour demander à l'utilisateur de choisir entre recherche par id ou par label
    Et lui demande de rentrer la valeur à rechercher.
    Définit également search_term comme variable globale pour tout le carnet.
    """
    #global search_term  # Déclare search_term comme variable globale

    print("🔍 Choisissez le type de recherche :")
    print("1️⃣  Recherche par ID (ex: Q42)")
    print("2️⃣  Recherche par label (ex: Douglas Adams)")

    choice = input("👉 Entrez 1 ou 2 : ").strip()

    if choice == "2":
        search_term = input("📝 Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
        if not search_term:
            print("❌ Aucun nom d'entité fourni.")
            exit(1)
        search_term = f'"{search_term}"'  # Guillemets pour recherche textuelle
        return search_term
    elif choice == "1":
        search_term = input("🆔 Entrez l'ID de l'entité à rechercher dans wikidata (ex: Q42) : ").strip()
        # Validation de l'ID de l'entité
        if not search_term or not re.match(r"^Q\d+$", search_term):
            print("❌ ID d'entité invalide. Veuillez entrer un ID valide (ex: Q42).")
            exit(1)
        return search_term  # ← Retourner juste l'ID sans préfixe
    else:
        print("⚠️  Choix invalide, veuillez réessayer.")
        return ask_query_term()
    
print("✅ Fonctions utilitaires configurées !")

# =================================================================================
# FONCTION UTILITAIRE POUR EXTRAIRE LES ID DE WIKIDATA
# ================================================================================
def clean_entity_id(entity_uri):
    """
    Extrait l'ID d'une entité à partir de son URI
    :param entity_uri: URI de l'entité (ex: "http://www.wikidata.org/entity/Q42'")
    :return: ID de l'entité (ex: "Q42") ou une chaîne vide si l'URI est vide
    """
    if not entity_uri:
        return ""
    return entity_uri.split("/")[-1] if "/" in entity_uri else entity_uri

#### Configuration du client SPARQL

In [ ]:
# 🛠️ FONCTIONS DE PARAMETRAGE DES REQUÊTES ET DU CARNET                       |


# =================================================================================
# FONCTION POUR CRÉER UN CLIENT SPARQL
# =================================================================================
def create_sparql_client():
    """
    Crée un client SPARQL pour interagir avec Wikidata
    :return: Instance de SPARQLWrapper configurée pour Wikidata
    """
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(REQUEST_TIMEOUT)
    return sparql

# =================================================================================
# FONCTION PRINCIPALE POUR EXÉCUTER UNE REQUÊTE SPARQL
# ================================================================================
def execute_sparql_query(query, max_retries=MAX_RETRIES, use_pagination=False, limit=None, max_results=None):
    """
    Exécute une requête SPARQL avec gestion des erreurs, rate limiting et pagination optionnelle
    :param query: La requête SPARQL à exécuter
    :param max_retries: Nombre maximum de tentatives en cas d'échec
    :param use_pagination: Si True, active la pagination automatique
    :param limit: Taille des pages pour la pagination (défaut: LOOP_LIMIT)
    :param max_results: Nombre maximum de résultats à récupérer (None = illimité)
    :return: Résultats de la requête ou une liste vide en cas d'échec
    """
    sparql = create_sparql_client()
    

    # Si search_term est déjà défini, on l'utilise
    global search_term  # Assure que search_term est accessible globalement
    if 'search_term' not in globals():
        search_term = None
    # Si search_term est déjà défini, on l'utilise
    if search_term is not None:
        print(f"🎯 Recherche en cours pour le terme : {search_term}")
        # Déterminer le type de recherche et formater correctement
        if search_term.startswith('Q'):
            # Recherche par ID - ajouter le préfixe wd:
            formatted_term = f"wd:{search_term}"
        else:
            # Recherche textuelle - utiliser tel quel
            formatted_term = search_term
        
        # Remplacer {{search_term}} dans la requête
        query = query.replace("{search_term}", formatted_term)
    else:
        print("🤔 Aucun terme fourni, demande à l'utilisateur...")
        # Si search_term n'est pas défini, on demande à l'utilisateur
        search_term = ask_query_term()
        print(f"🎯 Recherche en cours pour le terme : {search_term}")
        if search_term:
            # Déterminer le type de recherche et formater correctement
            if search_term.startswith('Q'):
                # Recherche par ID - ajouter le préfixe wd:
                formatted_term = f"wd:{search_term}"
            else:
                # Recherche textuelle - utiliser tel quel
                formatted_term = search_term
            
            # Remplacer {{search_term}} dans la requête
            query = query.replace("{search_term}", formatted_term)
        else:

            print("❌ Aucun terme fourni, abandon de la requête.")
            return []

    # print("."*100)
    # print(f"🔍 Exécution de la requête SPARQL :\n{query}\n")
    # print("."*100)

    # MODE SIMPLE SANS PAGINATION
    if not use_pagination:
        print("🚀 Mode simple activé (sans pagination)")
        for attempt in range(max_retries):
            try:
                print(f"📡 Tentative {attempt + 1}/{max_retries} - Envoi vers Wikidata...")
                sparql.setQuery(query)
                
                print("⏳ Attente de la réponse du serveur...")
                results = sparql.query().convert()
                
                result_count = len(results["results"]["bindings"])
                print(f"🎉 Requête réussie ! {result_count} résultats récupérés")
                
                
                return results["results"]["bindings"]
                
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée: {e}")
                if attempt < max_retries - 1:
                    wait_time = RATE_LIMIT_DELAY * (attempt + 2)
                    print(f"⏳ Nouvelle tentative dans {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"💥 Requête échouée définitivement après {max_retries} tentatives")
                    return []
        return []
    
    # MODE PAGINATION ACTIVE
    if limit is None:
        limit = LOOP_LIMIT
    
    all_results = []
    offset = 0
    
    print(f"📚 Mode pagination activé (limite={limit} par page)...")
    
    while True:
        paginated_query = f"{query.rstrip()} LIMIT {limit} OFFSET {offset}"
        print(f"📖 Page en cours - OFFSET {offset}, LIMIT {limit}")
        success = False
        bindings = []
        for attempt in range(max_retries):
            try:
                sparql.setQuery(paginated_query)
                results = sparql.query().convert()
                bindings = results["results"]["bindings"]
                success = True
                break
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée à l'offset {offset}: {e}")
                if attempt < max_retries - 1:
                    time.sleep(RATE_LIMIT_DELAY * (attempt + 2))
                else:
                    print(f"💥 Échec définitif après {max_retries} tentatives à l'offset {offset}")
                    return all_results  # Retourner ce qu'on a réussi à récupérer

        if not success:
            break

        if not bindings:
            print("🏁 Fin de la pagination - Plus de résultats disponibles")
            break

        all_results.extend(bindings)
        print(f"✅ Page récupérée : {len(bindings)} résultats (📊 Total: {len(all_results)})")
        
        if max_results and len(all_results) >= max_results:
            print(f"🎯 Limite de {max_results} résultats atteinte !")
            all_results = all_results[:max_results]
            break

        offset += limit
        print(f"⏸️  Pause de {RATE_LIMIT_DELAY}s avant la page suivante...")
        time.sleep(RATE_LIMIT_DELAY)
    
    print(f"🏆 Mission accomplie ! Total récupéré : {len(all_results)} résultats")
    return all_results

# =================================================================================
# FONCTION POUR EXÉCUTER DES REQUÊTES SPARQL EN BATCH : VERIFIER L'UTILITE
# ================================================================================
def execute_batch_queries(queries, description="Requêtes", use_pagination=False):
    """
    Exécute une liste de requêtes SPARQL en batch
    :param queries: Requête SPARQL unique ou liste de requêtes
    :param description: Description de la tâche pour le logging
    :param use_pagination: Si True, active la pagination pour chaque requête
    :return: Liste de tous les résultats combinés
    """
    # Vérifier si queries est une string ou une liste
    if isinstance(queries, str):
        # Si c'est une string, c'est une seule requête
        print(f"🔹 Exécution d'une requête unique: {description}")
        return execute_sparql_query(queries, use_pagination=use_pagination)
    
    # Si c'est une liste, traiter comme batch
    all_results = []
    for i, query in enumerate(tqdm(queries, desc=description)):
        results = execute_sparql_query(query, use_pagination=use_pagination)
        all_results.extend(results)
        if (i + 1) % BATCH_SIZE == 0:
            time.sleep(RATE_LIMIT_DELAY)
    return all_results

# =================================================================================
# FONCTION POUR EXÉCUTER UNE REQUÊTE SPARQL AVEC PAGINATION
# ================================================================================
def execute_paginated_query(base_query, limit=None, max_results=MAX_RESULTS, ask_term=False):
    """
    Fonction helper pour exécuter facilement une requête avec pagination
    """
      
    return execute_sparql_query(
        base_query, 
        use_pagination=True, 
        limit=limit, 
        max_results=max_results
    )
print("🎨 Fonctions de requête SPARQL configurées avec succès !")

## 📝 Préparation

### ❓ Définir le terme à chercher

In [269]:
search_term = ask_query_term()  # Demande le terme de recherche à l'utilisateur
print("🎯" + "="*98 + "🎯")
print(f"🔍 TERME DE RECHERCHE SÉLECTIONNÉ : {search_term}")

# # Si le search_term est un ID (commence par 'Q'), on fait une requête pour trouver son label, sinon on fait une recherche pour trouver l'id correspondant au label
# if search_term.startswith('Q'):
#     # Recherche par ID
#     print(f"🆔 Recherche de l'entité avec ID {search_term}...")
#     query = f"""SELECT ?itemLabel ?item WHERE {{
#         wd:{search_term} rdfs:label ?itemLabel.
#         BIND(wd:{search_term} AS ?item).
#         SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
#     }}"""
# else:
#     # Recherche par label
#     print(f"📝 Recherche de l'entité avec le label '{search_term}'...")
#     search_term = search_term.replace('"', '')  # Enlever les guillemets pour la requête
#     search_term = f'"{search_term}"'  # Ajouter des guillemets pour la recherche textuelle
#     # Requête pour trouver l'ID correspondant au label
#     query = f"""
#     SELECT ?itemLabel ?item WHERE {{
#     ?item  rdfs:label {search_term}@en.
#     SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
#     }}
#     """
# # Exécuter la requête SPARQL
# print("🎯" + "="*98 + "🎯")
# results = execute_sparql_query(query, use_pagination=False)
# if not results:
#     print("🎯" + "="*98 + "🎯")
#     print("❌ Aucun résultat trouvé pour ce terme.")
# else:
#     print("🎯" + "="*98 + "🎯")
#     print(f"🎉 {len(results)} résultat(s) trouvé(s) pour le terme '{search_term}' :")
#     for result in results:
#         item_id = clean_entity_id(result['item']['value'])
#         item_label = result['itemLabel']['value']
#         print(f"   📌 {item_label} ({item_id})")

🔍 Choisissez le type de recherche :
1️⃣  Recherche par ID (ex: Q42)
2️⃣  Recherche par label (ex: Douglas Adams)
🎯==================================================================================================🎯
🔍 TERME DE RECHERCHE SÉLECTIONNÉ : Q4828699
🎯==================================================================================================🎯
🔍 TERME DE RECHERCHE SÉLECTIONNÉ : Q4828699


### 📚 Bibliothèque de requêtes

In [270]:
# 📚 BIBLIOTHÈQUE DE REQUÊTES

# =================================================================================
# REQUETE POUR OBTENIR DANS QUELLES PROPRIÉTÉS L'ENTITÉ EST UTILISÉE
# =================================================================================

query_properties = """
SELECT ?prop WHERE {
  {
    SELECT ?prop 
           WHERE {
      ?item ?prop {search_term}.
    }
    GROUP BY ?prop
  }
}
"""

# =================================================================================
# REQUETE POUR OBTENIR LES PAGES LIEES A UN IDENTIFIANT RECHERCHÉ
# =================================================================================

query_pages_linked = """    
SELECT 
  ?item 
  ?itemLabel 
  (GROUP_CONCAT(DISTINCT ?propLabel; separator=", ") AS ?props)
  (COALESCE(?itemDescription_fr, ?itemDescription_en) AS ?itemDescription) # renvoie la première description valide trouvée.
  ?parent1
  (COALESCE(?parent1Label_fr, ?parent1Label_en) AS ?parent1Label)
  ?parent2
  (COALESCE(?parent2Label_fr, ?parent2Label_en) AS ?parent2Label)
WHERE {
  VALUES ?prop { 
    {values_block}
  }
  ?item ?prop  wd:{search_term} .
  OPTIONAL {
  ?wd wikibase:directClaim ?prop .
  ?wd rdfs:label ?propLabel .
  FILTER(LANG(?propLabel) = "fr")
  }
  # Description fr/en selon disponibilité
  OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
  OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }

  # parent1Label fr/en selon disponibilité
  OPTIONAL {
    ?item wdt:P31 ?parent1 .
    OPTIONAL { ?parent1 rdfs:label ?parent1Label_fr . FILTER(LANG(?parent1Label_fr) = "fr") }
    OPTIONAL { ?parent1 rdfs:label ?parent1Label_en . FILTER(LANG(?parent1Label_en) = "en") }

    # parent2Label fr/en selon disponibilité
    OPTIONAL {
      ?item wdt:P279 ?parent2 .
      OPTIONAL { ?parent2 rdfs:label ?parent2Label_fr . FILTER(LANG(?parent2Label_fr) = "fr") }
      OPTIONAL { ?parent2 rdfs:label ?parent2Label_en . FILTER(LANG(?parent2Label_en) = "en") }
    }
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en". }
}
GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?parent1Label_fr ?parent1Label_en ?parent2Label_fr ?parent2Label_en ?parent1 ?parent2
"""
print("📚 Bibliothèque de requêtes SPARQL chargée avec succès !")

📚 Bibliothèque de requêtes SPARQL chargée avec succès !


## 🔎 Requête

### Etape 1. Recherche des propriétés liées à un terme

In [271]:
# EXECUTER LA REQUÊTE

print("🔍 ÉTAPE 1 : Recherche des propriétés liées au terme")
print("📡 Envoi de la requête vers Wikidata...")

# Exécuter la requête
query_results = execute_sparql_query(query_properties)
print(f"✅ Recherche terminée - {len(query_results)} propriétés trouvées")

🔍 ÉTAPE 1 : Recherche des propriétés liées au terme
📡 Envoi de la requête vers Wikidata...
🎯 Recherche en cours pour le terme : Q4828699
🚀 Mode simple activé (sans pagination)
📡 Tentative 1/3 - Envoi vers Wikidata...
⏳ Attente de la réponse du serveur...
🎉 Requête réussie ! 26 résultats récupérés
✅ Recherche terminée - 26 propriétés trouvées
🎉 Requête réussie ! 26 résultats récupérés
✅ Recherche terminée - 26 propriétés trouvées


### Etape 2. Créer un dictionnaire avec index avec la liste de toutes les propriétés extraites

In [272]:
# EXTRAIRE LES IDs

print("🔧 ÉTAPE 2 : Extraction et formatage des identifiants de propriétés")

all_property_ids = [clean_entity_id(prop['prop']['value']).replace('#', ':') for prop in query_results if 'prop' in prop and 'value' in prop['prop']]

print(f"📋 Liste des propriétés extraites ({len(all_property_ids)}) :")
for i, prop_id in enumerate(all_property_ids, 1):
    print(f"   {i:2d}. {prop_id}")

print(f"\n✅ Extraction terminée : {len(all_property_ids)} propriétés prêtes pour la requête")

🔧 ÉTAPE 2 : Extraction et formatage des identifiants de propriétés
📋 Liste des propriétés extraites (26) :
    1. P2579
    2. P69
    3. P301
    4. owl:sameAs
    5. P106
    6. P31
    7. P5869
    8. P812
    9. P101
   10. P452
   11. P5869
   12. P2579
   13. P301
   14. P642
   15. about
   16. P31
   17. P4070
   18. P452
   19. P812
   20. P101
   21. P69
   22. P106
   23. P812
   24. P101
   25. P9488
   26. P9488

✅ Extraction terminée : 26 propriétés prêtes pour la requête


### Etape 3. Requêter toutes les pages reliées à l'identifiant recherché

In [273]:
# Génère une requête SPARQL pour trouver toutes les pages reliées au search_term via les propriétés de all_property_ids

def build_related_pages_query(search_term, property_ids):
    """
    Construit une requête SPARQL pour trouver toutes les pages reliées à search_term via une liste de propriétés.
    :param search_term: ID Wikidata (ex: Q42)
    :param property_ids: liste de propriétés (ex: ['P50', 'P170'])
    :return: requête SPARQL (str)
    """
    # Filtrer pour ne garder que les propriétés Wikidata valides (commençant par P suivi de chiffres)
    valid_properties = []
    invalid_properties = []
    
    for pid in property_ids:
        # Ne garder que les propriétés P suivies de chiffres
        if pid.startswith("P") and pid[1:].isdigit():
            valid_properties.append(f"wdt:{pid}")
        else:
            invalid_properties.append(pid)
    
    # Afficher les propriétés filtrées
    print(f"🔍 Propriétés valides trouvées : {len(valid_properties)}")
    if invalid_properties:
        print(f"⚠️  Propriétés non-Wikidata ignorées ({len(invalid_properties)}) : {invalid_properties[:5]}{'...' if len(invalid_properties) > 5 else ''}")
    
    if not valid_properties:
        print("❌ Aucune propriété Wikidata valide trouvée !")
        return None
    
    # Créer le bloc VALUES avec les propriétés valides
    values_block = " ".join(valid_properties)
    
    # Remplacer les placeholders dans la requête
    query = query_pages_linked.replace("{search_term}", search_term)
    query = query.replace("{values_block}", values_block)
    
    return query

print("🔍 ÉTAPE 3 : Construction et exécution de la requête des pages liées")
print("🏗️  Construction de la requête SPARQL...")

query = build_related_pages_query(search_term, all_property_ids)

if query is None:
    print("❌ Impossible de construire la requête - aucune propriété valide")
    related_pages_results = []
else:
    print("📡 Envoi de la requête principale vers Wikidata...")
    # Exécuter la requête pour obtenir les pages liées
    related_pages_results = execute_sparql_query(query, use_pagination=False)

🔍 ÉTAPE 3 : Construction et exécution de la requête des pages liées
🏗️  Construction de la requête SPARQL...
🔍 Propriétés valides trouvées : 24
⚠️  Propriétés non-Wikidata ignorées (2) : ['owl:sameAs', 'about']
📡 Envoi de la requête principale vers Wikidata...
🎯 Recherche en cours pour le terme : Q4828699
🚀 Mode simple activé (sans pagination)
📡 Tentative 1/3 - Envoi vers Wikidata...
⏳ Attente de la réponse du serveur...
🎉 Requête réussie ! 164 résultats récupérés
🎉 Requête réussie ! 164 résultats récupérés


## 📁 Export

In [274]:
# Exporter les résultats dans un fichier JSON

print("💾 EXPORT : Sauvegarde des résultats...")
print("📄 Création du fichier JSON...")

related_pages_json_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_pages_{search_term}.json"
related_pages_json_filepath = os.path.join(output_dir, related_pages_json_filename)
with open(related_pages_json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(related_pages_results, jsonfile, ensure_ascii=False, indent=2)
print(f"✅ Fichier JSON sauvegardé : 📁 {related_pages_json_filepath}")

💾 EXPORT : Sauvegarde des résultats...
📄 Création du fichier JSON...
✅ Fichier JSON sauvegardé : 📁 ./QUERY_BY_PROP_output\20250708_162944_pages_Q4828699.json


In [275]:
# Exporter les résultats dans un fichier CSV

print("📊 Création du fichier CSV...")

related_pages_csv_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_pages_{search_term}.csv"
related_pages_csv_filepath = os.path.join(output_dir, related_pages_csv_filename)
with open(related_pages_csv_filepath, 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['item', 'itemLabel', 'props', 'itemDescription', 'parent1', 'parent1Label', 'parent2', 'parent2Label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in related_pages_results:
        writer.writerow({
            'item': clean_entity_id(result.get('item', {}).get('value', '')),
            'itemLabel': result.get('itemLabel', {}).get('value', ''),
            'props': result.get('props', {}).get('value', ''),
            'itemDescription': result.get('itemDescription', {}).get('value', ''),
            'parent1': clean_entity_id(result.get('parent1', {}).get('value', '')),
            'parent1Label': result.get('parent1Label', {}).get('value', ''),
            'parent2': clean_entity_id(result.get('parent2', {}).get('value', '')),
            'parent2Label': result.get('parent2Label', {}).get('value', '')
        })

print(f"✅ Fichier CSV sauvegardé : 📁 {related_pages_csv_filepath}")
print("\n🎊 MISSION ACCOMPLIE ! Tous les fichiers ont été générés avec succès ! 🎊")
print(f"📂 Consultez le dossier : {output_dir}")

📊 Création du fichier CSV...
✅ Fichier CSV sauvegardé : 📁 ./QUERY_BY_PROP_output\20250708_162944_pages_Q4828699.csv

🎊 MISSION ACCOMPLIE ! Tous les fichiers ont été générés avec succès ! 🎊
📂 Consultez le dossier : ./QUERY_BY_PROP_output
